### Import standard GDAL and NumPy packages

In [1]:
import numpy as np
from osgeo import gdal

### Import skope.RasterDataset class

In [2]:
from skope import RasterDataset

### Set the location and name of the data set files to create

In [3]:
test_dataset_filename = '../data/annual_5x5x5_dataset_float32_variable.tif'
test_dataset_uncertainty_filename = '../data/annual_5x5x5_dataset_float32_variable_uncertainty.tif'

### Create the test dataset and uncertainty files using a local utility function

In [4]:
dataset = RasterDataset.create(test_dataset_filename,
                               file_format='GTiff',
                               pixel_type=gdal.GDT_Float32, 
                               shape=(5, 5, 5),
                               origin=(-123, 45),
                               pixel_size=(1.0, 1.0))

dataset_uncertainty = RasterDataset.create(test_dataset_uncertainty_filename,
                                           file_format='GTiff',
                                           pixel_type=gdal.GDT_Float32, 
                                           shape=(5, 5, 5),
                                           origin=(-123, 45),
                                           pixel_size=(1.0, 1.0))

### Define the pixel values to be assigned to the first band of the data set and uncertainties

In [5]:
import math
nodata = math.nan

band_1_data = np.array([
    [100, 101.1, 102.2, 103.3, 104.4],
    [110, 111.1, 112.2, 113.3, 114.4],
    [120, 121.1, 122.2, 123.3, 124.4],
    [130, 131.1, 132.2, 133.3, nodata],
    [140, 141.1, 142.2, 143.3, 144.4]
])

band_1_uncertainty = np.array([
    [10, 10.1, 10.2, 10.3, 10.4],
    [11, 11.1, 11.2, 11.3, 11.4],
    [12, 12.1, 12.2, 12.3, 12.4],
    [13, 13.1, 13.2, 13.3, 13.4],
    [14, 14.1, 14.2, 14.3, 14.4], 
]) 

### Write pixel values to each band of the data set
For all bands other than the first, the value of each pixel is 100 more than the pixel directly below it in the previous band.

In [6]:
for i in range(0,5):
    
    dataset.write_band(
        band_index=i,
        array=(band_1_data + i * 100),
        nodata=nodata
    )
    
    dataset.write_pixel(band_index=i, row=3, column=4, value=nodata)

    dataset_uncertainty.write_band(
        band_index=i,
        array=(band_1_uncertainty + i * 10),
        nodata=nodata
    )
    
dataset.write_pixel(band_index=2, row=2, column=4, value=nodata)
    
dataset.flush()
dataset_uncertainty.flush()

### Confirm some expected pixel values in the new data set and uncertainties

In [7]:
assert dataset.value_at_pixel(band_index=0,row=0,column=0) == 100
assert dataset.value_at_pixel(band_index=0,row=4,column=4) - 144.4 < 0.001

assert math.isnan(dataset.value_at_pixel(band_index=0, row=3, column=4))
assert math.isnan(dataset.value_at_pixel(band_index=1, row=3, column=4))
assert math.isnan(dataset.value_at_pixel(band_index=2, row=3, column=4))
assert math.isnan(dataset.value_at_pixel(band_index=3, row=3, column=4))
assert math.isnan(dataset.value_at_pixel(band_index=4, row=3, column=4))
assert math.isnan(dataset.value_at_pixel(band_index=2, row=2, column=4))

### Display pixel values and uncertainties for each band of new data set for easy reference

In [8]:
for i in range(0,5):
    print("\nband index", i, "\n", 
          dataset.read_band(i), "\n", 
          "\nband", i, "uncertainty\n", 
          dataset_uncertainty.read_band(i))


band index 0 
 [[100.  101.1 102.2 103.3 104.4]
 [110.  111.1 112.2 113.3 114.4]
 [120.  121.1 122.2 123.3 124.4]
 [130.  131.1 132.2 133.3   nan]
 [140.  141.1 142.2 143.3 144.4]] 
 
band 0 uncertainty
 [[10.  10.1 10.2 10.3 10.4]
 [11.  11.1 11.2 11.3 11.4]
 [12.  12.1 12.2 12.3 12.4]
 [13.  13.1 13.2 13.3 13.4]
 [14.  14.1 14.2 14.3 14.4]]

band index 1 
 [[200.  201.1 202.2 203.3 204.4]
 [210.  211.1 212.2 213.3 214.4]
 [220.  221.1 222.2 223.3 224.4]
 [230.  231.1 232.2 233.3   nan]
 [240.  241.1 242.2 243.3 244.4]] 
 
band 1 uncertainty
 [[20.  20.1 20.2 20.3 20.4]
 [21.  21.1 21.2 21.3 21.4]
 [22.  22.1 22.2 22.3 22.4]
 [23.  23.1 23.2 23.3 23.4]
 [24.  24.1 24.2 24.3 24.4]]

band index 2 
 [[300.  301.1 302.2 303.3 304.4]
 [310.  311.1 312.2 313.3 314.4]
 [320.  321.1 322.2 323.3   nan]
 [330.  331.1 332.2 333.3   nan]
 [340.  341.1 342.2 343.3 344.4]] 
 
band 2 uncertainty
 [[30.  30.1 30.2 30.3 30.4]
 [31.  31.1 31.2 31.3 31.4]
 [32.  32.1 32.2 32.3 32.4]
 [33.  33.1 33.2 33